In [3]:
import pandas as pd
import numpy as np
import re

from plotnine import *
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer

In [4]:
df = pd.read_csv("../data/cyberbullying_tweets.csv")

In [5]:
df.shape

(47692, 2)

In [25]:
#https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python
def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    vTEXT = re.sub('@[^\s]+','', vTEXT)
    return(vTEXT)

In [26]:
#remove links 
df['no_links_text'] = [remove_urls(i) for i in df['tweet_text']]

#character and word length
df['char_len'] = [len(i) for i in df.no_links_text]
df['word_count'] = [len(i.split()) for i in df.no_links_text]


df['contains_link'] = np.where(df['tweet_text'].str.contains('http'),True,False)
df['is_retweet'] = np.where(df['tweet_text'].str.contains('RT'),True,False)


df.head(5)

,tweet_text,cyberbullying_type,no_links_text,char_len,word_count,contains_link,is_retweet
0,"In other words #katandandre, your food was cra...",not_cyberbullying,"In other words #katandandre, your food was cra...",61,9,False,False
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,Why is #aussietv so white? #MKR #theblock #ImA...,115,14,False,False
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,a classy whore? Or more red velvet cupcakes?,45,8,False,False
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,"meh. :P thanks for the heads up, but not too...",93,17,False,False
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,This is an ISIS account pretending to be a Ku...,89,17,False,False


In [28]:
df = pd.read_csv(r'C:\Users\Dask\Downloads\cleaned_tweets_sample.csv')

In [29]:
X = df['no_links_text']
y = df['cyberbullying_type']

In [ ]:
counter vectorizer
TFID
sbert

#me
SVM
forest

#cassidy
NB
Logreg

f1 score and confusion matrix